In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM customer
LIMIT 5

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [4]:
%%sql
WITH all_p_usa AS (
SELECT 
g.name name,
sum(i.invoice_id) total_pur
FROM track t
INNER JOIN invoice_line il ON il.track_id=t.track_id
INNER JOIN invoice i ON i.invoice_id = il.invoice_id
INNER JOIN customer c ON c.customer_id = i.customer_id
INNER JOIN genre g on g.genre_id=t.genre_id
WHERE c.country = 'USA'
GROUP BY g.name
)

SELECT 
name name_genre,
total_pur abs_number,
round(CAST(total_pur as FLOAT)/(SELECT sum(total_pur) FROM all_p_usa)*100,2) || '%' percentages
FROM all_p_usa
ORDER by 2 DESC

Done.


name_genre,abs_number,percentages
Rock,172638,51.76%
Alternative & Punk,43711,13.11%
Metal,43537,13.05%
Alternative,14191,4.25%
R&B/Soul,11393,3.42%
Pop,10684,3.2%
Blues,10049,3.01%
Latin,6134,1.84%
Hip Hop/Rap,5821,1.75%
Easy Listening,4656,1.4%


In [5]:
%%sql
SELECT
e.first_name || ' ' || e.last_name employee_name,
e.hire_date,
round(sum(total),0) sum_pursh,
count(total) count_pursh,
round(avg(total),1) avg_pursh 
FROM employee e
INNER JOIN customer c ON c.support_rep_id = e.employee_id
INNER JOIN invoice i ON i.customer_id = c.customer_id

WHERE title = 'Sales Support Agent'

GROUP BY 1

Done.


employee_name,hire_date,sum_pursh,count_pursh,avg_pursh
Jane Peacock,2017-04-01 00:00:00,1732.0,212,8.2
Margaret Park,2017-05-03 00:00:00,1584.0,214,7.4
Steve Johnson,2017-10-17 00:00:00,1394.0,188,7.4


In [6]:
%%sql
WITH BD AS (
SELECT
CASE
WHEN (SELECT count(*) FROM customer where country = c.country) = 1 THEN "Other"
ELSE c.country
END AS country,
c.customer_id,
il.*
FROM invoice_line il
INNER JOIN invoice i ON i.invoice_id = il.invoice_id
INNER JOIN customer c ON c.customer_id = i.customer_id
)

SELECT 
Country,
count(distinct customer_id) numbers_of_customer,
round(sum(unit_price),1) value_of_sales,
round(sum(unit_price)/count(distinct customer_id),3) avg_value_per_customer,
round(sum(unit_price)/count(distinct invoice_id),3) avg_order_value
FROM BD
GROUP BY 1
ORDER BY 3 DESC


Done.


country,numbers_of_customer,value_of_sales,avg_value_per_customer,avg_order_value
Other,15,1094.9,72.996,7.449
USA,13,1040.5,80.038,7.943
Canada,8,535.6,66.949,7.047
Brazil,5,427.7,85.536,7.011
France,5,389.1,77.814,7.781
Germany,4,334.6,83.655,8.161
Czech Republic,2,273.2,136.62,9.108
United Kingdom,3,245.5,81.84,8.769
Portugal,2,185.1,92.565,6.384
India,2,183.2,91.575,8.721


In [7]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    ),
magic_case AS (
SELECT
    ifs.*,
    CASE
    WHEN (
        SELECT t.track_id FROM track t
        WHERE t.album_id = (
            SELECT t2.album_id FROM track t2
            WHERE t2.track_id = ifs.first_track_id) 

        EXCEPT 

        SELECT il2.track_id FROM invoice_line il2
        WHERE il2.invoice_id = ifs.invoice_id
    ) IS NULL
    AND (
        SELECT il2.track_id FROM invoice_line il2
        WHERE il2.invoice_id = ifs.invoice_id

        EXCEPT 

        SELECT t.track_id FROM track t
        WHERE t.album_id = (
            SELECT t2.album_id FROM track t2
            WHERE t2.track_id = ifs.first_track_id
        ) 
    ) IS NULL
    THEN "yes"
    ELSE "no"
    END AS "album_purchase"
    FROM invoice_first_track ifs
)


SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM magic_case
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674
